In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.utils import np_utils
from keras.layers.core import Dense, Dropout, Activation
#from keras.layers import Flatten
#from sklearn.metrics import roc_auc_score
#from io import StringIO
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import keras_metrics

/Users/dave/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 载入数据

In [2]:
df = pd.read_csv('/Users/dave/Desktop/毕设数据/df.csv')
df25 = pd.read_csv('/Users/dave/Desktop/毕设数据/df25.csv')
del df['Unnamed: 0']
del df25['Unnamed: 0']

In [3]:
# df = df[:2053]
# df25 = df25[:2053]

In [4]:
df[0:1]

,state4,state3,reqInterTime,downTraffic,downTrafficDiff,downTrafficDiffAbs
0,1,1,0.0,101363,101363,101363


In [7]:
len(df)

75595

In [6]:
df25[0:1]

,state4,state3,reqInterTime,downTraffic,downTrafficDiff,downTrafficDiffAbs
0,1,1,0.0,252044,252044,252044


# 将特征进行归一化

In [70]:
df_scaled = preprocessing.normalize(df[['reqInterTime','downTraffic','downTrafficDiff','downTrafficDiffAbs']])
df25_scaled = preprocessing.normalize(df25[['reqInterTime','downTraffic','downTrafficDiff','downTrafficDiffAbs']])

df_train = pd.DataFrame(df_scaled, columns = ['reqInterTime','downTraffic','downTrafficDiff','downTrafficDiffAbs']) 
df_test = pd.DataFrame(df25_scaled, columns = ['reqInterTime','downTraffic','downTrafficDiff','downTrafficDiffAbs']) 
df_train['label'] = list(df['state4'])
df_test['label'] = list(df25['state4'])

cols = list(df_train)
cols.insert(0, cols.pop(cols.index('label')))
df_train = df_train.loc[:, cols]

cols = list(df_test)
cols.insert(0, cols.pop(cols.index('label')))
df_test = df_test.loc[:, cols]

#df_train['label'] = df_train['label'].replace([1,2,3,4],[0.1,0.2,0.3,0.4])
#df_test['label'] = df_test['label'].replace([1,2,3,4],[0.1,0.2,0.3,0.4])

In [71]:
df_train[0:1]

,label,reqInterTime,downTraffic,downTrafficDiff,downTrafficDiffAbs
0,1,0.0,0.57735,0.57735,0.57735


In [72]:
df_test[0:1]

,label,reqInterTime,downTraffic,downTrafficDiff,downTrafficDiffAbs
0,1,0.0,0.57735,0.57735,0.57735


# 构造监督数据集

In [93]:
train = df_train.values.astype('float16')
test = df_test.values.astype('float16')
#设置“输入数据回看”的步数
look_back = 4
trainX = []
testX = []
trainY = []
testY = []

for i in range(len(train)-look_back-1):
    traintmp = train[i:(i+look_back),1:]
    trainX.append(traintmp)
    trainY.append(train[i+look_back,0])
    
trainX = np.reshape(trainX, (len(trainX), look_back, 4))
trainY = np.reshape(trainY, (len(trainY),1))

for i in range(len(test)-look_back-1):
    testmp = test[i:(i+look_back),1:]
    testX.append(testmp)
    testY.append(test[i+look_back,0])
    
testX = np.reshape(testX, (len(testX), look_back, 4))
testY = np.reshape(testY, (len(testY),1))

In [94]:
print(trainX.shape,trainY.shape)
print(trainX[0:2])
print(trainY[0:2])

(75590, 4, 4) (75590, 1)
[[[0.00e+00 5.77e-01 5.77e-01 5.77e-01]
  [3.81e-06 5.77e-01 5.77e-01 5.77e-01]
  [3.81e-06 5.77e-01 5.77e-01 5.77e-01]
  [3.46e-06 5.77e-01 5.77e-01 5.77e-01]]

 [[3.81e-06 5.77e-01 5.77e-01 5.77e-01]
  [3.81e-06 5.77e-01 5.77e-01 5.77e-01]
  [3.46e-06 5.77e-01 5.77e-01 5.77e-01]
  [3.46e-06 5.77e-01 5.77e-01 5.77e-01]]]
[[1.]
 [1.]]


In [95]:
print(testX.shape,testY.shape)
#print(testX[0:1])
#print(testY[0:2])

(2395, 4, 4) (2395, 1)


In [96]:
#多类别标签二值化——onehot编码
# trainY_binary = preprocessing.label_binarize(trainY, classes = [1,2,3,4])
# testY_binary = preprocessing.label_binarize(testY, classes = [1,2,3,4])

In [97]:
from keras.utils.np_utils import to_categorical
testY_binary = to_categorical(testY, num_classes=None)
trainY_binary = to_categorical(trainY, num_classes=None)

In [98]:
testY_binary = np.array([i[:4] for i in testY_binary])
testY_binary

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [99]:
trainY_binary = np.array([i[:4] for i in trainY_binary])
trainY_binary

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [100]:
from keras.callbacks import Callback
from sklearn.metrics import f1_score, precision_score, recall_score

In [101]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        #val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_predict = np.argmax(np.asarray(self.model.predict(self.validation_data[0])), axis=1)
        #val_targ = self.validation_data[1]
        val_targ = np.argmax(self.validation_data[1], axis=1)
        _val_f1 = f1_score(val_targ, val_predict, average='macro')
        _val_recall = recall_score(val_targ, val_predict, average='macro')
        _val_precision = precision_score(val_targ, val_predict, average='macro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        #print('— val_f1: %f — val_precision: %f — val_recall %f' %(_val_f1, _val_precision, _val_recall))
        return

In [102]:
trainX.shape

(75590, 4, 4)

In [103]:
new_trainX = np.concatenate((trainX, np.zeros(shape = (trainX.shape[0],trainX.shape[1],1))),axis = 2)
new_testX = np.concatenate((testX, np.zeros(shape = (testX.shape[0],testX.shape[1],1))),axis = 2)

In [104]:
metrics = Metrics()

model = Sequential()
#model.add(LSTM(128, dropout = 0.2, activation='tanh', input_shape=(trainX.shape[1],trainX.shape[2])))
model.add(LSTM(128, input_shape=(new_trainX.shape[1], new_trainX.shape[2])))
#model.add(Dense(trainY.shape[1], activation='relu'))
model.add(Dense(4, activation='softmax'))
#model.add(Dense(1))
model.compile(
            loss = 'categorical_crossentropy', 
            optimizer = 'adam', 
            metrics = ['accuracy'])

#绘制acc-loss曲线
#history.loss_plot('epoch')

In [126]:
num_epoch = 2
batch = 16
for _ in range(num_epoch):
    for i in range(int(trainX.shape[0]//batch)):
        predict_feature = model.predict(new_trainX[i*batch:(i+1)*batch]).reshape(batch,new_trainX.shape[1],1)
        input_trainX = np.concatenate((trainX[i*batch:(i+1)*batch], predict_feature),axis=2)
        input_trainY_binary = trainY_binary[i*batch:(i+1)*batch]
        new_trainX[i*batch:(i+1)*batch] =  input_trainX
        
        predict_feature = model.predict(new_testX).reshape(new_testX.shape[0],new_testX.shape[1],1)
        input_testX = np.concatenate((testX, predict_feature),axis=2)
        new_testX =  input_testX
        
        model.fit(input_trainX, input_trainY_binary, class_weight = 'auto', epochs = 1, batch_size = batch, \
                   verbose = 1, validation_data = (input_testX, testY_binary), callbacks = [metrics])

Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 2.1252 - acc: 0.0000e+00 - val_loss: 0.4967 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 1.8085 - acc: 0.3125 - val_loss: 0.4967 - val_acc: 0.7194


/home/zdf/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zdf/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.1937 - acc: 1.0000 - val_loss: 0.4966 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.2028 - acc: 1.0000 - val_loss: 0.4965 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.3326 - acc: 1.0000 - val_loss: 0.4963 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.1996 - acc: 1.0000 - val_loss: 0.4961 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.2037 - acc: 1.0000 - val_loss: 0.4958 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 6ms/step - loss: 0.2026 -

Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0681 - acc: 1.0000 - val_loss: 0.5503 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0688 - acc: 1.0000 - val_loss: 0.5535 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0761 - acc: 1.0000 - val_loss: 0.5568 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.0538 - acc: 1.0000 - val_loss: 0.5601 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.0532 - acc: 1.0000 - val_loss: 0.5634 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0551 -

Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.0146 - acc: 1.0000 - val_loss: 0.7025 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.0146 - acc: 1.0000 - val_loss: 0.7053 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0144 - acc: 1.0000 - val_loss: 0.7080 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0135 - acc: 1.0000 - val_loss: 0.7108 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.0134 - acc: 1.0000 - val_loss: 0.7135 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.0167 -

Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 9ms/step - loss: 0.0052 - acc: 1.0000 - val_loss: 0.8430 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.0063 - acc: 1.0000 - val_loss: 0.8458 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 8ms/step - loss: 0.0168 - acc: 1.0000 - val_loss: 0.8488 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0085 - acc: 1.0000 - val_loss: 0.8519 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0046 - acc: 1.0000 - val_loss: 0.8549 - val_acc: 0.7194
Train on 16 samples, validate on 2395 samples
Epoch 1/1
16/16 [==============================] - 0s 7ms/step - loss: 0.0053 -

KeyboardInterrupt: 

In [44]:
score = model.evaluate(new_testX, testY_binary, batch_size = 16)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test precision:', metrics.val_precisions[-1])
print('Test recall:', metrics.val_recalls[-1])

2395/2395 [==============================] - 0s 66us/step
Test loss: 0.06692123617549156
Test accuracy: 0.8492693110647181
Test precision: 0.5440143526317004
Test recall: 0.7289868790036939


In [ ]:
help(LSTM)

In [ ]:
history

# 召回、f1分数

In [ ]:
#softmax结果转换成onehot编码
def props_to_onehot(props):
    if isinstance(props, list):
        props = np.array(props)
    a = np.argmax(props, axis=1)
    b = np.zeros((len(a), props.shape[1]))
    b[np.arange(len(a)), a] = 1
    return b

In [ ]:
#调用sklearn
predictions_last_epoch = model.predict(testX, batch_size = 16, verbose=1)
predictions_last_epoch = props_to_onehot(predictions_last_epoch)
target_names = ['CL', 'SW', 'CE', 'EM']
print('\n')
print(classification_report(testY_binary, predictions_last_epoch, target_names = target_names))

In [ ]:
print((len(testY_binary)),len(predictions_last_epoch))

In [ ]:
fig = plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc = 'upper left')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'lower left')

In [ ]:
preData = [(np.argmax(one_hot)+1) for one_hot in predictions_last_epoch]
trueData = [(np.argmax(one_hot)+1) for one_hot in testY_binary] 
time = np.arange(0,2395)

In [ ]:
print((len(preData)),len(trueData))

In [ ]:
# 统计两个数字中，相同位置值不同的个数
# 把预测错的值取出来
n=0
num = []
preDataWrong = []
trueDataTrue = []
for i in range(0,len(preData)):
    if preData[i] != trueData[i]:
        num.append(i)
        preDataWrong.append(preData[i])
        trueDataTrue.append(trueData[i])
        n+=1

In [ ]:
print(n)
print(num)
print(preDataWrong)
print(trueDataTrue)

In [ ]:
plt.plot(time,preData,'r.',time,trueData,'b-')
plt.draw()
plt.savefig("predict",dpi=600)
plt.show()

In [ ]:
import plotly as py
import plotly.graph_objs as go

In [ ]:
pyplt = py.offline.plot
predict = go.Scatter(
    x = time, 
    y = preData,
    name = 'predict'
)
true = go.Scatter(
    x = time, 
    y = trueData,
    mode = 'markers',
    name = 'true'
)
predictWrong = go.Scatter(
    x = num, 
    y = preDataWrong,
    mode = 'markers',
    name = 'predictWrong'
)
trueValue  = go.Scatter(
    x = num, 
    y = trueDataTrue,
    mode = 'markers',
    name = 'trueValue'
)

data = [predict, true, predictWrong, trueValue]
layout = go.Layout(
    title = 'Y轴',
    yaxis = dict(
        title = 'predict'
    ),
    yaxis2 = dict(
        title = 'true',
        color = '#ffa631',
        anchor = 'free',
        overlaying = 'y',
        side = 'left',
        position = 0.04 
    ),
    yaxis3 = dict(
        title = 'predictWrong',
        color = '#75878a',
        anchor = 'x',
        overlaying = 'y',
        side = 'right'
    ),
     yaxis4 = dict(
        title = 'trueValue',
        color = '#00e500',
        anchor = 'free',
        overlaying = 'y',
        side = 'right',
        position = 0.95
    )
)
fig = go.Figure(data = data, layout=layout)
plot_url = pyplt(fig, filename='ceshi.html')